In [3]:
pip install langchain-community

  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 2.4 MB/s eta 0:00:01
   ---------------- ----------------------- 1.0/2.5 MB 2.1 MB/s eta 0:00:01
   ------------------------ --------------- 1.6/2.5 MB 2.1 MB/s eta 0:00:01
   --------------------------------- ------ 2.1/2.5 MB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 2.3 MB/s eta 0:00:00
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from flask import Flask, request, jsonify
from langchain_community.document_loaders import RecursiveUrlLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA

# Step 1: Extract Data from Brainlox Website
def load_documents(url):
    loader = RecursiveUrlLoader(url=url, max_depth=2)
    documents = loader.load()
    return documents

# Step 2: Generate Embeddings and Store in FAISS Vector Store
def create_vector_store(documents):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_store = FAISS.from_documents(documents, embeddings)
    return vector_store

# Initialize the Vector Store with Website Data
documents = load_documents("https://brainlox.com/courses/category/technical")
vector_store = create_vector_store(documents)

# Initialize the QA Model using Llama 2 from Hugging Face
llm = HuggingFaceHub(repo_id="meta-llama/Llama-2-7b-chat-hf", model_kwargs={"temperature": 0.7, "max_length": 200})

# Create a Retrieval-Based QA Chain
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vector_store.as_retriever())

# Initialize Flask App
app = Flask(__name__)

@app.route('/chat', methods=['POST'])
def chat():
    user_query = request.json.get('query')
    if not user_query:
        return jsonify({'error': 'No query provided'}), 400

    # Get the response from the QA chain
    response = qa_chain.run(user_query)
    return jsonify({'response': response})

if __name__ == '__main__':
    app.run(debug=True)

C:\Users\pavan\AppData\Local\Temp\ipykernel_2180\1537934551.py:17: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


RuntimeError: Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.